# Setting Up TCLab

## Hardware Setup

![](figures/TCLab_labelled.jpg)

1. Insert the temperature control shield into the Arduino module. This will fit together in only one way. Squeeze the modules together until the shield is fully seated.

1. Connect the Arduino to your laptop with the USB data cable. You will need to USB-C to USB-A adapter if your laptop is equipped with USB-C only.

1. Plug the DC power adapter into a wall socket and connect the power cable to to the temperature control shield. **Note: The temperature control shield requires its own power supply. There are two places where the power connector will fit. It's important to be sure it is plugged into the shield, not the Arduino.**

TCLab requires the one-time installation of custom firmware on an Arduino device. The firmware is normally preinstalled when you receive the device. But if necessary, the firmware and instructions for installation are available from the [TCLab-Sketch repository](https://github.com/jckantor/TCLab-sketch).


## Software Setup

### Requirements

The tclab library must run locally on your laptop to access the USB port. A Python development system, such as [Anaconda](https://www.anaconda.com/products/individual) needs to be installed on your laptop prior to installing tclab.

### How the software is organized

Software for the Temperature Control Lab is organized as shown in the accompanying diagram. Note that the Python scripts must run on your laptop, not a remote server, in order to access the local USB port.

![](figures/TCLab_overview.png)

**_Jupyter notebooks and Python scripts:_**
The top level consists of the you code you write to implement control algorithms. This may be done in Jupyter/Python notebooks or directly in Python using an interactive development environment (IDE). This repository contains many examples of code written in Jupyter/Python notebooks.

**_TCLab.py:_**
[TCLab.py](https://github.com/jckantor/TCLab) is contained in a Python library entitled `tclab`. The library includes

* `TCLab()` class that creates an object to access to the device,
* `clock` for synchronizing with a real time clock
* `Historian()` class to create objects for data logging.
* `Plotter()` class to visualize data in real time.

**_TCLab-sketch:_**
The [TCLab-sketch](https://github.com/jckantor/TCLab-sketch) repository provides firmware to ensure intrisically safe operation of the Arduino board and shield. The sketch is downloaded to the Arduino using the [Arduino IDE](https://www.arduino.cc/en/Main/Software). Loading firmware to the Arduino is a one-time operation. 

**_Arduino:_**
The hardware platform for the Temperature Control Laboratory. The Python tools and libraries have been tested with the Arduino Leonardo boards.

### Installing the tclab library

The tclab library is installed from a terminal window (MacOS) or command window (PC) with the command

    pip install tclab

Alternatively, the installation can be performed from within a Jupyter/Python notebook with the command

In [6]:
!pip install tclab

There are occasional updates to the library. These can be installed by appending a ` --upgrade` to the above commands and demonstrated in the next cell.

In [7]:
!pip install tclab --upgrade

## Testing

The next cells test for some commonly encountered issues with installation and operation of the Temperature Control Laboratory. This approach uses the library [`ipytest`](https://github.com/chmp/ipytest) which adapts the well known `pytest` unit testing environment for use within Jupyter notebooks. If you have not encountered this before, unit testing is a widely used software testing technique to check for correct operation of individual units of code.

If the any of unit tests fail, comments within the unit test code may help determine the cause of the failure.

In [41]:
try:
    import ipytest
except:
    !pip install ipytest
    import ipytest
    
ipytest.autoconfig()

In [65]:
%%ipytest --verbosity=1

# Verify tclab has been installed and is accessible by the Python kernal.
    
def test_tclab_install():
    from tclab import TCLab, clock, Historian, Plotter
    
# Verify that TCLab can be run offline (i.e., without access to hardware).

def test_tclab_offline():
    from tclab import setup
    TCLab = setup(connected=False, speedup=20)
    with TCLab() as lab:
        pass
    
# TCLab cannot access the Arduino if it is run remotely on Google Colab. This is a common
# error since notebooks are so easy to open in Google Colab. The following test fails if
# the test is run on Google Colab.

def test_not_google_colab():
    import sys
    assert not "google.colab" in sys.modules
    
# The following tests require the TCLab hardware to be attached and operational

# Verify that a connection an be opened to the Arduino.

def test_tclab_connect():
    from tclab import TCLab, clock, Historian, Plotter
    lab = TCLab()
    lab.close()
    
# Verify tclab-sketch firmware version number is 2.0.1 or greater

def test_tclab_firmware_version():
    import packaging
    from tclab import TCLab
    with TCLab() as lab:
        vers = re.search(r'\s*([\d.]+)', lab.version).group(1)
    assert packaging.version.parse(vers) >= packaging.version.parse("2.0.1")

======================================= test session starts ========================================
platform darwin -- Python 3.9.7, pytest-6.2.4, py-1.10.0, pluggy-0.13.1 -- /Users/jeff/opt/anaconda3/bin/python
cachedir: .pytest_cache
rootdir: /Users/jeff/Google Drive/GitHub/cbe30338-book/tclab
plugins: anyio-2.2.0
collecting ... collected 5 items

tmpn552ce7j.py::test_tclab_install PASSED                                                    [ 20%]
tmpn552ce7j.py::test_tclab_offline PASSED                                                    [ 40%]
tmpn552ce7j.py::test_not_google_colab PASSED                                                 [ 60%]
tmpn552ce7j.py::test_tclab_connect PASSED                                                    [ 80%]
tmpn552ce7j.py::test_tclab_firmware_version PASSED                                           [100%]

======================================== 5 passed in 8.89s =========================================


In [46]:
from tclab import TCLab

lab = TCLab()
lab.LED(50)
lab.close()

TCLab version 0.4.9
Arduino Leonardo connected on port /dev/cu.usbmodem141101 at 115200 baud.
TCLab Firmware 2.0.1 Arduino Leonardo/Micro.
TCLab disconnected successfully.


## Troubleshooting

1. If your laptop consistently fails to locate or connect to the Arduino device, check the following:

    * Be sure you are have installed Python and tclab on your laptop, and that you are not attempting to run tclab on a remote server such as Google Colab. tclab requires direct access to the USB port on your laptop.
    * Check cable connections. The USB cable from the Arduino should be connected to your laptop. The USB cable for the heater shield should be connected to a USB power supply. Reversing these connections will not harm anything, but the system will not function.
    * 
    * The Arduino must have the tclab-sketch firmware to communicate with tclab. 
    * Though rare, we have experienced failed Arduino devices. Try changing Arduinos.

1. If you observe no change in temperature after the heaters on turned on:

    * Check that your heater power supply is plugged into power and into the top heater shield, not the Arduino.
    * If you're using a power strip, verify the power strip is turned on.
    * The LED indicator should be at 100% if the heaters are on.
    * Check that lab.P1 and/or lab.P2 are set at non-zero values between 0 and 255. Values of about 100 are good for testing.
    * We have experienced occasional failed USB power supplies. Try a different USB power supply.
